In [1]:
from html2image import Html2Image

hti = Html2Image(custom_flags=[
    '--default-background-color=00000000',
    '--hide-scrollbars',
    '--headless',
])
hti.screenshot(html_file='templates/template_coop.html', css_file='templates/styles_coop.css', save_as='temp.png')

/Users/tamino/Projects/hslu/dspro/dspro2-swiss-receipt-extractor/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
209260 bytes written to file /Users/tamino/Projects/hslu/dspro/dspro2-swiss-receipt-extractor/receipt_generation/temp.png


['/Users/tamino/Projects/hslu/dspro/dspro2-swiss-receipt-extractor/receipt_generation/temp.png']

In [2]:
from PIL import Image
import os

im = Image.open("temp.png")
im = im.crop(im.getbbox())

os.remove("temp.png")

In [3]:
import numpy as np
import numpy.random as rng

rng.seed(42)

image_array = np.array(im, dtype=np.float32)

background_color = np.array([255, 255, 255, 255])

# create a background canvas to generate a noise from
background = np.ones(image_array.shape)
background[:, :] = background_color

# generate a noise with floating point values representing the translucency of the background color
noise_dim = (image_array.shape[0], image_array.shape[1])
noise = rng.normal(-5, 20, noise_dim)
noise = np.round(np.clip(noise, 0, 50) / 50 * 255)

# apply noise as opacity to the background to create a noise in the color of the background
background[:, :, 3] = noise[:, :]

# add printing error lines
for _ in range(rng.randint(0, 6)):
    line_pos = rng.randint(0, image_array.shape[1] + 1)
    background[:, line_pos, :] = background_color

# convert noise overlay to an image
noise_overlay = Image.fromarray(background.astype(np.uint8))

In [6]:
# add the noise layer with transparency on top of the receipt image
noisy_image = Image.alpha_composite(im, noise_overlay)

# make the colors more natural by scaling them to be between 75-200
noisy_image = Image.fromarray((np.array(noisy_image, dtype=np.float32) / 255.0 * 165.0 + 35.0).astype(np.uint8))

noisy_image.save('out.png')

In [7]:
import cv2
import numpy as np

img = cv2.imread("out.png", cv2.IMREAD_COLOR)
background = cv2.imread('images/background.jpg', cv2.IMREAD_COLOR)

background = cv2.resize(background, (2376, 1584))
background = background[150:150 + 1100, 750:750+800]

width = img.shape[1]
height = img.shape[0]

pos_from = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
pos_to = np.float32([[200, 50], [600, 50], [50, 1000], [750, 1000]])

matrix = cv2.getPerspectiveTransform(pos_from, pos_to)
result = cv2.warpPerspective(img, matrix, (800, 1100), background, borderMode=cv2.BORDER_TRANSPARENT)

cv2.imwrite('transformed.png', result)

True